In [1]:
import pandas as pd
import subprocess,os
import HTSeq

# Setup

We will be using the HTSeq-count package for this version of the "count reads" pipeline.  
Add the location of your GFF file and the output directory for the final counts, as HTSeq-count outputs counts to a file.

In [2]:
GFF_FILE = 'example/ref/NC_000913.3.gff'

In [3]:
import pandas as pd

In [4]:
OUT_DIR = 'example/counts/'

In [5]:
DF_files = pd.read_csv('example/aligned_files.csv',index_col=0)

# Load BAM information

In [6]:
DF_counts = pd.DataFrame()
for i,row in DF_files.iterrows():
    bamfile = row.BAM
    outname = OUT_DIR+row.sample_id+'.txt'
    if not os.path.isfile(outname):
        with open(outname,'w') as outfile:
            subprocess.call(['htseq-count','-f','bam','-s','reverse','-r','pos',bamfile,GFF_FILE],stdout=outfile)
    # Write reads to dataframe
    df = pd.read_csv(outname,sep='\t',header=None,index_col=0,names=[row.sample_id])
    DF_counts = pd.concat([DF_counts,df],axis=1)
    
DF_counts = DF_counts.drop(['__no_feature','__ambiguous','__too_low_aQual','__not_aligned','__alignment_not_unique'])

In [7]:
# Remove genes with 0 counts over all conditions
DF_counts = DF_counts.loc[(DF_counts != 0).any(axis=1)]

In [8]:
# Save to file
DF_counts.to_csv(os.path.join(OUT_DIR,'counts.csv'))